In [ ]:
from LABNET import Teacher

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch
from collections import OrderedDict
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
import matplotlib.pyplot as plt

def plot_losses(losses, plot_size=(10, 5)):
    """
    Plot a list of losses and adjust the shape of the plot.

    Args:
        losses (list): List of loss values to be plotted.
        plot_size (tuple): Tuple specifying the size of the plot (width, height).

    Returns:
        None
    """
    # Set the plot size based on the provided tuple
    plt.figure(figsize=plot_size)
    
    # Create a line plot of the losses
    plt.plot(losses, label='Loss', color='b')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss Over Epochs')
    plt.legend()
    
    # Show the plot
    plt.show()

In [ ]:



layer_sizes = [8, 7,7,4]  # Inputs: 4, Hidden layers: [8, 8], Outputs: 3
teacher = Teacher(layer_sizes)



In [ ]:
teacher.configure()

In [ ]:
n = 5000 #number of samples to generate
teacher.generate_data(
    "train"
    ,n
    ,'normal'
    , m =0.0
    , std=1.0
)

In [ ]:
teacher.train_inputs.shape

In [ ]:
teacher.generate_data(
    "val"
    ,n
    ,'normal'
    , m =0.0
    , std=1.0
)

In [ ]:
teacher.val_inputs.shape

In [ ]:
teacher.model.state_dict()
#torch.save(neural_network.model.state_dict(), 'model_weights.pth')

In [ ]:
# Define the model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.hidden_1 = nn.Linear(8, 7) ##make these all variables duh
        self.hidden_2 = nn.Linear(7,7)
        #self.hidden_3 = nn.Linear(7,7)
        #self.hidden_4 = nn.Linear(7,7)
        #self.hidden_5 = nn.Linear(7,7)
        self.output = nn.Linear(7, 4)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.hidden_1(x))
        x = self.relu(self.hidden_2(x))
        #x = self.relu(self.hidden_3(x))
        #x = self.relu(self.hidden_4(x))
        #x = self.relu(self.hidden_5(x))
        x = self.output(x)
        return x

In [ ]:
student = MyModel()
print(student)

In [ ]:
##STARTING with the right answuers gives some close to zero row_comp:3.5974587e-08,0.0,0.0; 0.0,2.9802322e-08,1.4901161e-08
#interesting that you get a zero, and then a close to zero.  
#perturb_weights(mymodel, std_dev = 0.005) # converges
#perturb_weights(mymodel, std_dev = 0.009) # converges
#perturb_weights(mymodel, std_dev = 0.05) doesn't converge
#perturb_weights(mymodel, std_dev = 0.01) #converges
#perturb_weights(mymodel, std_dev = 0.01) #not converging well, even with lower lr, 0.003. 1t 0.001


# scale_weights(mymodel, scaling_factor = 1.1) ## converges, weights cross.  
#scale_weights(mymodel, scaling_factor = 1.5) ## good validation, non zero row comapre still..

#scale_weights(mymodel, scaling_factor = 2) ## good validation, row compare stays pretty close to the same...
## scale_weights(mymodel, scaling_factor = 1.05) row compare barely improves!!
student.state_dict()

In [ ]:
# Training loop
num_epochs = 100
batch_size = 32
learning_rate = 0.00025 #was 0.001, 0.0005
momentum = 0.95
criterion = nn.MSELoss()
optimizer = optim.Adam(student.parameters(), lr=learning_rate)

optimizer = optim.SGD(student.parameters(), lr=learning_rate, momentum=momentum)

In [ ]:
data = list(zip(teacher.train_inputs, teacher.train_targets))
input_tensors = torch.stack([torch.Tensor(x[0]) for x in data])
target_tensors = torch.stack([torch.Tensor(x[1]) for x in data])
dataset = TensorDataset(input_tensors, target_tensors)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
#conventional train (uses generated data once.)
print_every = 10

losses = []  # List to store losses

for epoch in range(num_epochs):
    total_loss = 0
    for input_batch, target_batch in dataloader:
        optimizer.zero_grad()  # Zero the gradients
        output = student(input_batch)  # Forward pass
        loss = criterion(output, target_batch)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update the weights
        total_loss += loss.item()

    # Print the average loss for this epoch
    avg_loss = total_loss / len(dataloader)
    losses.append(avg_loss)
    if (epoch + 1) % print_every == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.8f}')

In [ ]:
###train with no repeating training data
print_every = 10
batch_eternal = 50
data_n = 5000
num_epochs = 400
losses = []  # List to store losses

for epoch in range(num_epochs):
    total_loss = 0
    teacher.generate_data(
    "train"
    ,data_n
    ,'normal'
    , m =0.0
    , std=1.0
    )
    data = list(zip(teacher.train_inputs, teacher.train_targets))
    input_tensors = torch.stack([torch.Tensor(x[0]) for x in data])
    target_tensors = torch.stack([torch.Tensor(x[1]) for x in data])
    dataset = TensorDataset(input_tensors, target_tensors)
    dataloader = DataLoader(dataset, batch_size=batch_eternal, shuffle=True)
    
    for input_batch, target_batch in dataloader:
        optimizer.zero_grad()  # Zero the gradients
        output = student(input_batch)  # Forward pass
        loss = criterion(output, target_batch)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update the weights
        total_loss += loss.item()

    # Print the average loss for this epoch
    avg_loss = total_loss / len(dataloader)
    losses.append(avg_loss)
    if (epoch + 1) % print_every == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.8f}')

In [ ]:
plot_losses(losses, plot_size=(10, 5))

In [ ]:
v_batch_size = 10
val_data = list(zip(teacher.val_inputs, teacher.val_targets))
val_input_tensors = torch.stack([torch.Tensor(x[0]) for x in val_data])
val_target_tensors = torch.stack([torch.Tensor(x[1]) for x in val_data])
val_dataset = TensorDataset(val_input_tensors, val_target_tensors)
val_dataloader = DataLoader(val_dataset, batch_size=v_batch_size, shuffle=True)

In [ ]:
student.eval()

differences = []

with torch.no_grad():
    for val_batch in val_dataloader:
        val_inputs, val_targets = val_batch
        #val_inputs, val_targets = val_inputs\, val_targets.to(device)  # If you are using GPU, move data to the GPU
        # Forward pass
        val_outputs = student(val_inputs)
        
        # Calculate the absolute differences between predicted and actual values
        diff = val_outputs - val_targets
        
        # Extend the list of differences
        differences.extend(diff.cpu().numpy().flatten())  # Convert to NumPy for histogram plotting

In [ ]:
# Create a histogram of the differences
plt.hist(differences, bins=50)  # You can adjust the number of bins as needed
plt.xlabel('Absolute Differences')
plt.ylabel('Frequency')
plt.title('Histogram of Absolute Differences between Predicted and Actual Values')
plt.show()

In [ ]:
mean_difference = np.mean(differences)
std_deviation = np.std(differences)
mean_difference,std_deviation

In [ ]:
#what if it never sees the same data? don't need new val data, just new training data.  

#compare all novel data to epochs, that is, say we do 100 epochs on a dataset of 1000 input examples.  vs 100 epochs of all novel data.  which performs better on a hold out set?

#is there any value to swapping between momentum and just adam? messing with the lr as well?